## Import Library

In [219]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import re 
import string
import nltk
from nltk.corpus import stopwords
import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

## Dataset access to Big Query Data

In [220]:
client = bigquery.Client()
to_refer = client.dataset("stackoverflow", project="bigquery-public-data")

data = client.get_dataset(to_refer)

Using Kaggle's public dataset BigQuery integration.


In [221]:
sql = """
        SELECT id, title , tags 
        FROM 
        `bigquery-public-data.stackoverflow.stackoverflow_posts`
        WHERE
        title NOT LIKE '%None%' AND 
        (tags LIKE '%|python|%' OR tags LIKE '%|c#|%' OR
        tags LIKE '%|java|%' OR tags LIKE '%|r|%' OR
        tags LIKE '%|android|%' OR tags LIKE '%|html|%' OR
        tags LIKE '%|c++|%' OR tags LIKE '%|sql|%' OR
        tags LIKE '%|c|%' OR tags LIKE '%kotlin%') AND 
        LENGTH(tags) < 20
        LIMIT
        10000;
        """

df = client.query(sql).to_dataframe()

df.head()

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,id,title,tags
0,35493392,How to make a search result Page with a button...,javascript|html|jsp
1,10623932,Refer to one single SQL query 3 times on a pag...,php|sql|forms
2,8850977,MySQL before trigger,mysql|sql|triggers
3,9626410,How do I cast the result of a UNION in SQLite?,c++|sql|sqlite
4,26137459,Initialize AudioClip Object at Runtime Unity C...,c#|android|unity3d


## Preprocessing

**Stopwords**

In [222]:
df.title  = df.title.str.lower()
stop_words = set(stopwords.words('english'))

df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df.head()

,id,title,tags
0,35493392,make search result page button result item ope...,javascript|html|jsp
1,10623932,refer one single sql query 3 times page create...,php|sql|forms
2,8850977,mysql trigger,mysql|sql|triggers
3,9626410,cast result union sqlite?,c++|sql|sqlite
4,26137459,initialize audioclip object runtime unity c# a...,c#|android|unity3d


**Punctuation and hyperlinks except # and +**

In [223]:
punctuations = list(string.punctuation)
punctuations.remove("#")
punctuations.remove("+")

df['title'] = df['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (punctuations)]))
df.head()

,id,title,tags
0,35493392,make search result page button result item ope...,javascript|html|jsp
1,10623932,refer one single sql query 3 times page create...,php|sql|forms
2,8850977,mysql trigger,mysql|sql|triggers
3,9626410,cast result union sqlite?,c++|sql|sqlite
4,26137459,initialize audioclip object runtime unity c# a...,c#|android|unity3d


**Convert Tags string column to list column**

In [224]:
df['tags'] = df['tags'].str.split('|')
df.head()

,id,title,tags
0,35493392,make search result page button result item ope...,"[javascript, html, jsp]"
1,10623932,refer one single sql query 3 times page create...,"[php, sql, forms]"
2,8850977,mysql trigger,"[mysql, sql, triggers]"
3,9626410,cast result union sqlite?,"[c++, sql, sqlite]"
4,26137459,initialize audioclip object runtime unity c# a...,"[c#, android, unity3d]"


## MultiLabel Binarizer

In [225]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df['tags'])

y = multilabel_binarizer.transform(df['tags'])
y, y.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 (10000, 1558))

## Train Test Split

In [226]:
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.2)
x_train = x_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
x_train.shape, x_test.shape

((8000, 3), (2000, 3))

## Tokenizing and applying Pad Sequence

In [227]:
train_tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>')
train_tokenizer.fit_on_texts(x_train['title'].values)
train_word_index = train_tokenizer.word_index

In [228]:
vocab_length = len(train_word_index) + 1
vocab_length

6059

In [229]:
train_sequences = train_tokenizer.texts_to_sequences(x_train['title'].values)
len(train_sequences)

8000

In [230]:
test_sequences = train_tokenizer.texts_to_sequences(x_test['title'].values)
len(test_sequences)

2000

In [231]:
longest_sentence = len(max(train_sequences, key=len))
train_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=longest_sentence, truncating='post')
test_padded_seqeunces = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=longest_sentence, truncating='post')
longest_sentence

18

## LSTM

In [232]:
tf.keras.backend.clear_session()

embedding_dim = 50
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, embedding_dim, input_length=longest_sentence),
    
   tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(100, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LSTM(50, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(16),
    tf.keras.layers.Dense(y.shape[1], activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 50)            302950    
_________________________________________________________________
dropout (Dropout)            (None, 18, 50)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 18, 100)           60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 100)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 18, 100)           400       
_________________________________________________________________
lstm_1 (LSTM)                (None, 18, 50)            30200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 18, 50)            0

In [233]:
history = model.fit(train_padded_seqeunces, y_train, epochs=20, 
                    validation_data=(test_padded_seqeunces, y_test))

Epoch 1/20
250/250 [==============================] - 5s 10ms/step - loss: 0.0587 - accuracy: 0.1150 - val_loss: 0.0867 - val_accuracy: 0.1240
Epoch 2/20
250/250 [==============================] - 2s 7ms/step - loss: 0.0080 - accuracy: 0.1260 - val_loss: 0.0103 - val_accuracy: 0.1240
Epoch 3/20
250/250 [==============================] - 2s 8ms/step - loss: 0.0079 - accuracy: 0.1300 - val_loss: 0.0084 - val_accuracy: 0.1240
Epoch 4/20
250/250 [==============================] - 2s 7ms/step - loss: 0.0077 - accuracy: 0.1273 - val_loss: 0.0081 - val_accuracy: 0.1655
Epoch 5/20
250/250 [==============================] - 2s 8ms/step - loss: 0.0071 - accuracy: 0.1277 - val_loss: 0.0072 - val_accuracy: 0.1255
Epoch 6/20
250/250 [==============================] - 2s 8ms/step - loss: 0.0066 - accuracy: 0.1211 - val_loss: 0.0069 - val_accuracy: 0.1115
Epoch 7/20
250/250 [==============================] - 2s 7ms/step - loss: 0.0063 - accuracy: 0.1324 - val_loss: 0.0071 - val_accuracy: 0.1295
Epoch

## Predicting Tags

In [234]:
pred_prob = model.predict(test_padded_seqeunces)
t = 0.3
test_pred = (pred_prob >= t).astype(int)

In [235]:
y_test.shape

(2000, 1558)

In [236]:
for i in range(10):
    k = x_test.sample(1).index[0]
    print("Title: ", x_test['title'][k])
    print("Actual Tag: ", multilabel_binarizer.inverse_transform(y_test[k].reshape(1, -1))[0])
    print("Predicted tag: ", multilabel_binarizer.inverse_transform(test_pred[k].reshape(1, -1))[0])
    print('\n')

Title:  wrap word div?
Actual Tag:  ('css', 'html', 'html-lists')
Predicted tag:  ('css', 'html', 'javascript', 'jquery')


Title:  program terminated signal 25 file size limit exceeded; why?
Actual Tag:  ('c', 'c++', 'linux')
Predicted tag:  ('c', 'c++')


Title:  ensure php/html login form load values username password fields
Actual Tag:  ('forms', 'html', 'php')
Predicted tag:  ('html', 'php')


Title:  collision two boxes:vizard:python
Actual Tag:  ('c', 'python', 'vizard')
Predicted tag:  ('android', 'java', 'python')


Title:  equivalent rectangle (java) android
Actual Tag:  ('android', 'java', 'rect')
Predicted tag:  ('android', 'java')


Title:  recently switch sqlite mysql. need convert code c
Actual Tag:  ('api', 'c', 'mysql', 'sqlite')
Predicted tag:  ('sql',)


Title:  mysql select count sum inner join group
Actual Tag:  ('group-by', 'mysql', 'sql')
Predicted tag:  ('join', 'mysql', 'sql')


Title:  getting records connected getting last one
Actual Tag:  ('android', 'sql', 